In [84]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import h5py
import numpy as np
# import jax
# import jax.numpy as jnp
# import haiku as hk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import initializers

np.random.seed(8)
tf.random.set_seed(8)

In [50]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
f = h5py.File('/clusterfs/ml4hep/mfong/transfer_learning/delphes_train.h5', 'r')
f2 = h5py.File('/clusterfs/ml4hep/mfong/transfer_learning/delphes_test.h5', 'r')

In [4]:
feature_keys = ['fjet_clus_eta', 'fjet_clus_phi', 'fjet_clus_pt']
for k in f.keys():
    print(k, f[k].shape)

fjet_clus_E (4000068, 200)
fjet_clus_eta (4000068, 200)
fjet_clus_phi (4000068, 200)
fjet_clus_pt (4000068, 200)
fjet_eta (4000068,)
fjet_m (4000068,)
fjet_phi (4000068,)
fjet_pt (4000068,)
labels (4000068,)


In [55]:
x = np.concatenate([f[k] for k in feature_keys], axis=1)
x.shape

(4000068, 600)

In [57]:
scaler = StandardScaler()
scaler.fit(x)

x = scaler.transform(x)

In [63]:
y = f["labels"][:]
y.shape

(4000068,)

## Logistic Regression

In [66]:
# quick logistic regression test
from sklearn.linear_model import LogisticRegression

In [103]:
logistic_reg = LogisticRegression(random_state=8)
logistic_reg.fit(x[:256000], y[:256000])

/global/home/users/mfong/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=8)

In [108]:
print("In sample Acc:", logistic_reg.score(x[:256000], y[:256000]))
print("Out of sample Acc:", logistic_reg.score(x[256000:512000], y[256000:512000]))

In sample Acc: 0.77969921875
Out of sample Acc: 0.7800078125


## Fully Connected NN

In [85]:
model = Sequential()
model.add(Dense(64, input_shape=(600,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [86]:
model.weights

[<tf.Variable 'dense_18/kernel:0' shape=(600, 64) dtype=float32, numpy=
 array([[-0.02558209, -0.03079722,  0.016824  , ..., -0.02682649,
          0.08155291,  0.03427844],
        [-0.0223344 ,  0.00111546, -0.03407578, ..., -0.0214681 ,
         -0.02278921, -0.01171391],
        [ 0.04239799, -0.04694584, -0.04062798, ..., -0.06666273,
         -0.04073998, -0.03917032],
        ...,
        [-0.03037461,  0.09354982,  0.08570632, ..., -0.07042781,
          0.07279515,  0.00742079],
        [ 0.07771151, -0.06429571,  0.06354998, ..., -0.08217992,
         -0.03654257,  0.02611994],
        [ 0.08519252, -0.07327612, -0.06033182, ..., -0.03859762,
          0.05293804, -0.03686836]], dtype=float32)>,
 <tf.Variable 'dense_18/bias:0' shape=(64,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [91]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [92]:
model.predict(x[:10])

array([[0.48095438],
       [0.38764784],
       [0.46399817],
       [0.45152295],
       [0.3950969 ],
       [0.22237472],
       [0.33510858],
       [0.54572433],
       [0.20901929],
       [0.3356871 ]], dtype=float32)

In [93]:
model.fit(x[:256000], y[:256000], epochs=100, batch_size=256)

Epoch 1/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4576 - accuracy: 0.7801
Epoch 2/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4400 - accuracy: 0.7902
Epoch 3/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4367 - accuracy: 0.7926
Epoch 4/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4344 - accuracy: 0.7939
Epoch 5/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4332 - accuracy: 0.7944
Epoch 6/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4313 - accuracy: 0.7955
Epoch 7/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4301 - accuracy: 0.7959
Epoch 8/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4291 - accuracy: 0.7965
Epoch 9/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4278 - accuracy: 0.7972
Epoch 10/100
1000/1000 [==============================] - 2s 2ms/step - l

1000/1000 [==============================] - 2s 2ms/step - loss: 0.3836 - accuracy: 0.8209
Epoch 80/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3831 - accuracy: 0.8212
Epoch 81/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3834 - accuracy: 0.8201
Epoch 82/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3829 - accuracy: 0.8208
Epoch 83/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3823 - accuracy: 0.8215
Epoch 84/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3820 - accuracy: 0.8213
Epoch 85/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3820 - accuracy: 0.8210
Epoch 86/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3820 - accuracy: 0.8211
Epoch 87/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3815 - accuracy: 0.8217
Epoch 88/100
1000/1000 [==============================] - 2s 2ms/step - loss:

In [101]:
loss, accuracy = model.evaluate(x[256000:512000], y[256000:512000])
print("Accuracy: ", accuracy)
print("loss: ", loss)

8000/8000 [==============================] - 13s 2ms/step - loss: 0.4778 - accuracy: 0.7833
Accuracy:  0.7833476662635803
loss:  0.47781795263290405


In [102]:
model.weights

[<tf.Variable 'dense_18/kernel:0' shape=(600, 64) dtype=float32, numpy=
 array([[-0.86564416, -0.00440815,  0.15640804, ..., -0.08163399,
         -0.48209044, -0.4300402 ],
        [-0.88983804,  0.25214803,  0.3447045 , ..., -0.0623154 ,
         -0.25372744, -0.5870787 ],
        [-0.44380027,  0.12676288, -0.15944123, ...,  0.04876088,
         -0.10257276, -0.46758392],
        ...,
        [ 0.31976184,  0.83458143,  0.71953225, ...,  0.4022435 ,
          0.51563746,  0.5399129 ],
        [ 0.41662648,  0.7033081 ,  0.69598955, ...,  0.38785124,
          0.40374252,  0.561602  ],
        [ 0.4242242 ,  0.69245523,  0.57545984, ...,  0.44767445,
          0.4853984 ,  0.50967455]], dtype=float32)>,
 <tf.Variable 'dense_18/bias:0' shape=(64,) dtype=float32, numpy=
 array([-0.29801795, -1.6214861 , -0.97545475,  0.1046082 , -0.53778446,
        -1.5176228 , -1.7269754 , -0.29097793, -1.160995  , -0.18192871,
        -0.17721514, -0.52237654, -0.12061778, -0.44242588, -0.2944851 ,
